In [ ]:
import numpy as np
import pandas as pd

# 1. Load raw data (only finalized columns) -------------------------------
dtypes = {'id':'int64', 'item_nbr':'int32', 'store_nbr':'int8', 'onpromotion':'str'}
train = pd.read_csv('/content/drive/MyDrive/Dissertation/all files/train.csv',
                    dtype=dtypes,
                    parse_dates=['date'],
                    usecols=['id','date','store_nbr','item_nbr','unit_sales','onpromotion'])

stores = pd.read_csv('/content/drive/MyDrive/Dissertation/all files/stores.csv',
                     usecols=['store_nbr','city','state','type','cluster'])

items = pd.read_csv('/content/drive/MyDrive/Dissertation/all files/items.csv',
                    usecols=['item_nbr','family','class','perishable'])

hol = pd.read_csv('/content/drive/MyDrive/Dissertation/all files/holidays_events.csv',
                  dtype={'transferred':'str'},
                  parse_dates=['date'],
                  usecols=['date','locale','locale_name','type','transferred'])

In [ ]:
# 2. Prepare holiday/event flags ------------------------------------------
hol = (
    hol[~hol.transferred.str.lower().eq('true')]      # drop transferred
       .query("type!='Work Day'")                     # drop compensatory work days
       .assign(
           on_hol=lambda df: df.type.map({
               'Holiday':'Holiday','Bridge':'Holiday','Additional':'Holiday'
           }),
           on_evt=lambda df: df.type.map({'Event':'Event'})
       )
)
locL = (
    hol.query("locale=='Local'")
       .loc[:, ['date','locale_name','on_hol','on_evt']]
       .rename(columns={'locale_name':'city'})
)
locR = (
    hol.query("locale=='Regional'")
       .loc[:, ['date','locale_name','on_hol','on_evt']]
       .rename(columns={'locale_name':'state'})
)
locN = hol.query("locale=='National'")[['date','on_hol','on_evt']]

In [ ]:
# 3. Merge into single DataFrame -----------------------------------------
df = (
    train
      .merge(stores, on='store_nbr', how='left')
      .merge(items,  on='item_nbr',  how='left')
      .merge(locL,   on=['date','city'],  how='left')
      .merge(locR,   on=['date','state'], how='left')
      .merge(locN,   on='date',           how='left')
)

In [ ]:
# 4. Keep only final features --------------------------------------------
df = df[[
    'id','unit_sales','date','store_nbr','item_nbr',
    'city','state','type','cluster','family','class','perishable',
    'onpromotion','on_hol','on_evt'
]].copy()

In [ ]:
# 5. Basic transformations -----------------------------------------------
#  unit_sales: clip returns to 0,
df['unit_sales'] = df['unit_sales'].clip(lower=0)
#df['unit_sales'] = np.log1p(df['unit_sales'])

#  calendar features
df['month']       = df['date'].dt.month
df['wage']        = df['date'].dt.day.isin([15, 31]).astype(int)
df['is_weekend']  = (df['date'].dt.dayofweek >= 5).astype(int)

#  promotions & perishability
df['onpromotion'] = df['onpromotion'].map({'False': 0, 'True': 1}).fillna(2).astype(int)
df['perishable']  = df['perishable'].map({0: 1.0, 1: 1.25}).fillna(2)

#  holiday/event flags
# prefer map+fillna so the result is a pandas Series
df['on_hol'] = df['on_hol'].map({'Holiday': 1}).fillna(-1).astype(int)
df['on_evt'] = df['on_evt'].map({'Event':   1}).fillna(-1).astype(int)

In [ ]:
# 6. Outlier detection & replacement (IQR fence rule) -------------------

# Compute Q1, Q3, IQR and upper bound per item
Q1 = df.groupby('item_nbr')['unit_sales'].quantile(0.25)
Q3 = df.groupby('item_nbr')['unit_sales'].quantile(0.75)
IQR = Q3 - Q1
upper_bound = Q3 + 1.5 * IQR

# Summarize outlier rate per item
def detect_outlier_rate(group, ub):
    n = len(group)
    return (group['unit_sales'] > ub[group.name]).sum() / max(n, 1)

outlier_rate = df.groupby('item_nbr').apply(lambda g: detect_outlier_rate(g, upper_bound))
outlier_rate.name = 'outlier_rate'

# Compute non-outlier mean per item
non_outlier_mean = (
    df.groupby('item_nbr')['unit_sales']
      .apply(lambda x: x[x <= upper_bound[x.name]].mean())
)
non_outlier_mean.name = 'non_outlier_mean'

# Merge stats back into df
df = df.merge(Q1.rename('Q1'),           on='item_nbr')
df = df.merge(Q3.rename('Q3'),           on='item_nbr')
df = df.merge(upper_bound.rename('upper_bound'), on='item_nbr')
df = df.merge(outlier_rate.reset_index(),       on='item_nbr')
df = df.merge(non_outlier_mean.reset_index(),   on='item_nbr')

# Replace outliers for items with >10% outlier rate
mask = (df['outlier_rate'] > 0.10) & (df['unit_sales'] > df['upper_bound'])
df.loc[mask, 'unit_sales'] = df.loc[mask, 'non_outlier_mean']

In [ ]:
# Convert 'date' column to datetime
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# Create 'day' column with weekday names
df['day'] = df['date'].dt.day_name()

In [ ]:
# 7. Target (mean/rank) encode categoricals

categorical_cols = [
    'store_nbr','item_nbr','city','state',
    'type','cluster','family','class', 'day'
]
for col in categorical_cols:
    mean_sales = df.groupby(col)['unit_sales'].mean().sort_values()
    rank_map   = {cat: rank+1 for rank, cat in enumerate(mean_sales.index)}
    df[col + '_rank'] = df[col].map(rank_map)

In [ ]:
# 4. Keep only final features --------------------------------------------
df1 = df[[
    'id','unit_sales','date','day','store_nbr','item_nbr',
    'city','state','type','cluster','family','class','perishable',
    'onpromotion','on_hol','on_evt'
]]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Compute correlation matrix (as before)
corr = df1.select_dtypes(include=['number']).drop(columns=['id']).corr()

# Plot with annotations
plt.figure(figsize=(12, 12))
sns.heatmap(
    corr,
    annot=True,      # show numbers
    fmt='.2f',       # two decimal places
    cmap='coolwarm',
    square=True,
    cbar=True,
    linewidths=0.5   # grid lines between cells
)
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.title('Feature Correlation Matrix')
plt.tight_layout()
plt.show()


In [ ]:
# Create df_final with only the required features (dropping raw categoricals & outlier stats)
df_final = df[[
    'id',
    'unit_sales',
    'date',
    'perishable',
    'onpromotion',
    'on_hol',
    'on_evt',
    'month',
    'wage',
    'is_weekend',
    'day_rank',
    'store_nbr_rank',
    'item_nbr_rank',
    'city_rank',
    'state_rank',
    'type_rank',
    'cluster_rank',
    'family_rank',
    'class_rank'
]]

In [ ]:
# Drop '_rank' suffix from rank columns for clarity
df_final.columns = [col.replace('_rank', '') if col.endswith('_rank') else col for col in df_final.columns]

In [ ]:
df_final.count()

,0
id,124217119
unit_sales,124217119
date,124217119
perishable,124217119
onpromotion,124217119
on_hol,124217119
on_evt,124217119
month,124217119
wage,124217119
is_weekend,124217119


In [ ]:
output_path = '/content/drive/MyDrive/Dissertation/df_train.csv'
df_final.to_csv(output_path, index=False)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 1) Compute absolute correlations to unit_sales
#corr = df_final.select_dtypes(include=['number']).drop(columns=['id']).corr()
corr_to_target = corr['unit_sales'].abs().drop('unit_sales')
sorted_corr = corr_to_target.sort_values()

# 2) Identify bottom 5%
n_feats   = len(sorted_corr)
n_remove  = max(1, int(np.floor(n_feats * 0.05)))
worst_feats = sorted_corr.index[:n_remove]

# 3) Plot horizontal bar chart
plt.figure(figsize=(8, max(6, n_feats*0.2)))
bars = plt.barh(sorted_corr.index, sorted_corr.values)

# Hatch the worst 5%
for i, feat in enumerate(sorted_corr.index):
    if feat in worst_feats:
        bars[i].set_hatch('//')

plt.xlabel('Absolute correlation with unit_sales')
plt.ylabel('Feature')
plt.title('Feature Correlations with unit_sales  (bottom 5% hatched)')
plt.tight_layout()
plt.show()

print(f"Bottom 5% to drop ({n_remove} features): {list(worst_feats)}")


In [ ]:
#  Apply log1p transformation to target
df_final["unit_sales"] = df_final["unit_sales"].clip(lower=0)  # Ensure no negative values

#  Train-test split based on year
train_df = df_final[df_final['date'].dt.year < 2017]
test_df  = df_final[df_final['date'].dt.year == 2017]

# Drop 'date' and 'year' from features
X_train = train_df.drop(columns=["unit_sales", "date", "wage","on_evt"])
y_train = train_df["unit_sales"]

X_test = test_df.drop(columns=["unit_sales", "date", "wage","on_evt"])
y_test = test_df["unit_sales"]

X_train_pgrs = train_df.drop(columns=["date", "wage","on_evt"])
X_test_pgrs = test_df.drop(columns=["date", "wage","on_evt"])

In [ ]:
# Output path
output_path = '/content/drive/MyDrive/Dissertation/final/'

# Create and save DataFrames
df_20 = X_train_pgrs.iloc[:20_000_000]
df_20.to_csv(f"{output_path}df_20.csv", index=False)




In [ ]:
output_path = '/content/drive/MyDrive/Dissertation/final/'

df_40 = X_train_pgrs.iloc[:40_000_000]
df_40.to_csv(f"{output_path}df_40.csv", index=False)

df_60 = X_train_pgrs.iloc[:60_000_000]
df_60.to_csv(f"{output_path}df_60.csv", index=False)

df_80 = X_train_pgrs.iloc[:80_000_000]
df_80.to_csv(f"{output_path}df_80.csv", index=False)

In [ ]:
output_path = '/content/drive/MyDrive/Dissertation/final/df_train.csv'
X_train_pgrs.to_csv(output_path, index=False)

In [ ]:
output_path = '/content/drive/MyDrive/Dissertation/final/df_test.csv'
X_test_pgrs.to_csv(output_path, index=False)